In [2]:
import sys
print(sys.executable)


C:\Users\riya\AppData\Local\Programs\Python\Python312\python.exe


In [1]:
import folium
print("Folium imported successfully")


Folium imported successfully


In [1]:
# ============================================================
# MODULE 5: GEOSPATIAL MAPPING & HEATMAP VISUALIZATION
# Fully interactive with source filtering
# ============================================================

import pandas as pd
import folium
from folium.plugins import HeatMap

# -----------------------------
# Step 1: Load dataset
# Must contain: latitude, longitude, pollution_source, pm25, datetime_local, district
# -----------------------------
df = pd.read_csv("Final_clean_dataset_with_source.csv")

# Drop rows with missing coordinates or PM2.5
df = df.dropna(subset=['latitude', 'longitude', 'pm25', 'pollution_source', 'datetime_local'])

# Convert datetime column to datetime type for filtering
df['datetime_local'] = pd.to_datetime(df['datetime_local'])

# -----------------------------
# Step 2: Optional date filter
# Can be modified for dashboard selection
# -----------------------------
start_date = '2025-01-01'
end_date   = '2025-12-31'
df = df[(df['datetime_local'] >= start_date) & (df['datetime_local'] <= end_date)]

# -----------------------------
# Step 3: Initialize base map
# Centered around mean coordinates
# -----------------------------
base_map = folium.Map(
    location=[df['latitude'].mean(), df['longitude'].mean()],
    zoom_start=6,
    tiles="cartodbpositron"
)

# -----------------------------
# Step 4: Add PM2.5 heatmap (severity gradient)
# -----------------------------
heat_data = [[row['latitude'], row['longitude'], row['pm25']] for _, row in df.iterrows()]

HeatMap(
    heat_data,
    radius=15,
    blur=20,
    max_zoom=10,
    min_opacity=0.4
).add_to(base_map)

# -----------------------------
# Step 5: Overlay source-specific markers (interactive toggle)
# -----------------------------
source_icons = {
    "Industrial": "industry",
    "Vehicular": "car",
    "Burning": "fire",
    "Natural": "leaf"
}

source_colors = {
    "Industrial": "red",
    "Vehicular": "blue",
    "Burning": "orange",
    "Natural": "green"
}

# Create a separate FeatureGroup for each pollution source
for source in df['pollution_source'].unique():
    fg = folium.FeatureGroup(name=source)  # This allows toggle in LayerControl
    subset = df[df['pollution_source'] == source]

    for _, row in subset.iterrows():
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=f"""
            <b>Source:</b> {row['pollution_source']}<br>
            <b>PM2.5:</b> {row['pm25']}<br>
            <b>Location:</b> {row['district']}<br>
            <b>Date:</b> {row['datetime_local'].date()}
            """,
            icon=folium.Icon(
                color=source_colors.get(source, "gray"),
                icon=source_icons.get(source, "info-sign"),
                prefix="fa"
            )
        ).add_to(fg)
    
    # Add the FeatureGroup to the map
    fg.add_to(base_map)

# -----------------------------
# Step 6: Add LayerControl for source toggling
# -----------------------------
folium.LayerControl(collapsed=False).add_to(base_map)

# -----------------------------
# Step 7: Save map for dashboard integration
# -----------------------------
base_map.save("final_pollution_source_heatmap.html")

# -----------------------------
# Step 8: Summary
# -----------------------------
print("✅ Fully interactive pollution heatmap created successfully!")
print("📍 File saved as: final_pollution_source_heatmap.html")
print("\n📌 Summary:")
print(f"- Dataset used: {len(df)} locations")
print("- Features visualized: PM2.5 intensity & predicted pollution source")
print("- Interactivity: Toggle sources via LayerControl, click markers for details")
print("- High-risk zones visualized via heatmap color gradient")
print("- Ready for embedding in web dashboard")


✅ Fully interactive pollution heatmap created successfully!
📍 File saved as: final_pollution_source_heatmap.html

📌 Summary:
- Dataset used: 77994 locations
- Features visualized: PM2.5 intensity & predicted pollution source
- Interactivity: Toggle sources via LayerControl, click markers for details
- High-risk zones visualized via heatmap color gradient
- Ready for embedding in web dashboard
